In [1]:
import mdtraj as md
import glob as glob
from scipy import spatial
from collections import defaultdict
from tqdm import tqdm_notebook
import numpy as np
import os

In [ ]:
def clusterer(files, threshold=0.5, interchain=True, intraresidue=False, return_type=0, debug=True):
    
    """
    Function to calculate the number of contacts each residue has with neighbouring residues
    
    v0.1: First version, only has one parameter, the threshold setting
    v0.2: Vectorised distance calculation with pyml calculate_distance 
          (~100 times speed increase in distance calculation)
    v0.3: Fully vectorised distance calculation with scipy (~100x speed increase overall)
    v0.4: Added inter/intra chain option, inter/intra residue and return value type 
          (residue label or position or both)
    
    :type directory: list
    :type threshold: float
    :type interchain: bool
    :type intraresidue: bool
    :type return_type: int
    :type debug: bool
    
    :param directory: files to run calculation
    :param threshold: threshold in nm in contact definition
    :param interchain: whether to look at interchain contacts
    :param intraresidue: whether to look for interactions within the same residue
    :param return_type: 0 returns label and position, 1 returns only residue label and 2 returns only position
    :param debug: switch on debug mode
    
    :rtype: dict
    :return: a dict with dictionaries with the contacts for each each residue  
    
    Pseudocode:
    ----------
        1. for each file in directory:
            a. Read PDB to extract residue names
            b. Parse file with mdtraj
            c. initialise contacts dict
            d. Calculate dense matrix with euclidean distances and then convert to squareform
            f. for each position below threshold:
                i. Find out position of corresponding atom
               ii. store appropriate data
        2. Return dict (with key corresponding to file) with contacts of each file
    """
        
    result = dict()
    
    for file in tqdm_notebook(files):
        
        file_name = os.path.split(file)[-1].split('.')[0] # get file basename
                
        seq=list()

        atom_map = dict()

        with open(file, 'r') as f:
            file_length = 0
            i=0
            for line in f.readlines():

                line_el = line.split()


                if len(line_el) < 6:
                    continue

                try:
                    if line_el[2] not in ['N', 'O', 'CA', 'C']:
                        file_length += 1
                        atom_map[i] = (line_el[4], line_el[5], line_el[3])
                        i+=1

                    if (line_el[4], line_el[5]) not in seq:
                        seq.append((line_el[4], line_el[5]))
                except:
                    raise ValueError(line)

        result_i = dict()

        model = md.load_pdb(file)

        model = model.atom_slice(model.top.select('all and sidechain'))

        atoms = model.n_atoms

        contacts = defaultdict(set)

        dists = spatial.distance.pdist(model.xyz.squeeze(0))

        dist_threshold = np.where(spatial.distance.squareform(dists) < threshold)

        for atom1, atom2 in zip(dist_threshold[0], dist_threshold[1]):

            position1 = atom_map[atom1][0]+atom_map[atom1][1]
            position2 = atom_map[atom2][0]+atom_map[atom2][1]
            
            if not interchain and position1[0] != position2[0]:
                # if atoms are in different chains skip
                continue
                
            if not intraresidue and position1 == position2:
                # skip if atoms are in same residue
                continue
                
            if not isinstance(position1, str) or not isinstance(position2, str):
                continue

            if return_type == 0:
                contacts[position1].add((position2, atom_map[atom2][2]))
            elif return_type == 1:
                contacts[position1].add(atom_map[atom2][2])
            elif return_type == 2:
                contacts[position1].add(position2)
            else:
                raise ValueError("Unknown return type!")
                
        result[file_name] = contacts
                
    return result

In [ ]:
files = glob.glob('./models/*pdb')
result = clusterer(files, 0.5, debug=False)

A Jupyter Widget

In [ ]:
[x for x in result['P1']]

In [ ]:
aa_order = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [ ]:
three_letter ={'V':'VAL', 'I':'ILE', 'L':'LEU', 'E':'GLU', 'Q':'GLN',
'D':'ASP', 'N':'ASN', 'H':'HIS', 'W':'TRP', 'F':'PHE', 'Y':'TYR',
'R':'ARG', 'K':'LYS', 'S':'SER', 'T':'THR', 'M':'MET', 'A':'ALA',
'G':'GLY', 'P':'PRO', 'C':'CYS'}

In [ ]:
def vector_representation(data):
    """
    Function to represent cluster data in vector form
    
    :type data: dict
    
    :param data: data from clusterer
    
    :rtype: dict
    :return: returns dict with same keys as data containing a dict 
             with the vector for each position
             
    Explanation:
    ------------
    
    Each vector has the following order:
    
    [A, C, D, E, F, G, H, I, K, L, M, N, P, Q, R, S, T, V, W, Y]
    
    Each position either takes up the value of 0 (no contact) or 1 (contact)
    
    """
    
    result = dict()
    
    for key, value in tqdm_notebook(data.items()):
        
        file_result = dict()
        
        for pos, contact in value.items():
            
            res_list = [x[1] for x in contact] 
            file_result[pos] = [1 if three_letter[x] in res_list else 0 for x in aa_order]
            
        result[key] = file_result
                
    return result

In [ ]:
vectors = vector_representation(result)

In [ ]:
vectors['P1']

In [ ]:
h1_vectors = []
h1_labels = []
for file, data in vectors.items():
    if 'H1' in data:
        h1_vectors.append(data['H1'])
        h1_labels.append(file)

In [ ]:
spatial.distance.squareform(spatial.distance.pdist(h1_vectors, metric='cosine'))

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
clustered_data = linkage(y=spatial.distance.pdist(h1_vectors, metric='cosine'))

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(20,10))
dendrogram(clustered_data, labels=h1_labels)
plt.show()